# Benchmark Vanilla GP Supervised Learning

## Preamble

This notebook provides an example of how to perform Gaussian Process Regression on a 1D manifold. In this example we consider a supervised learning scenario, namely the number of labeled data points is equivalent to the number of the sampled points from the underlying manifold.

In [1]:
import torch
import gpytorch
import numpy as np
from time import time
from manifold_gp.models.vanilla_gp import VanillaGP

## Dataset Preprocessing

### Load & Settings

In [2]:
# bike, buzz_tomshardware, buzz_twitter, ctslices, elevators, protein, song, mnist, mnist_single
dataset = 'ctslices'
samples_split = 0.9
preprocess = False
normalize_features = False
normalize_labels = True

data = np.load('../datasets/'+dataset+'.npy')

x = data[:, :-1]
y = data[:, -1]

idx = np.random.permutation(x.shape[0])
x = x[idx, :]
y = y[idx]

num_samples = int(samples_split * len(data))
sampled_x, sampled_y = x[:num_samples, :], y[:num_samples]
test_x, test_y = x[num_samples:, :], y[num_samples:]

# # Map features to [-1, 1] (used in gpytorch)
# x = data[:, :-1]
# x = x - x.min(0)[0]
# x = 2 * (x/ x.max(0)[0]) - 1

In [3]:
if preprocess:
    # # remove coincident points
    # sampled_x, id_unique = np.unique(sampled_x, axis=0, return_index=True)
    # sampled_y = sampled_y[id_unique]

    # cut between x% and y% percentile of distances
    num_avg = 1
    p_start, p_end = 0.1, 0.9
    num_samples = sampled_x.shape[0]
    
    import faiss
    res = faiss.StandardGpuResources()
    knn = faiss.GpuIndexIVFFlat(res, sampled_x.shape[1], 1, faiss.METRIC_L2)
    knn.train(sampled_x)
    knn.add(sampled_x)
    v = np.sqrt(knn.search(sampled_x, num_avg+1)[0][:,1:])
    idx = np.argsort(v.mean(axis=1))
    idx = np.delete(idx, np.arange(int(num_samples*p_start),int(num_samples*p_end)))
    sampled_x = np.delete(sampled_x, idx, axis=0)
    sampled_y = np.delete(sampled_y, idx)
m = sampled_x.shape[0]

### Trainset & Testset

In [4]:
train_split = int(0.75 * m)

train_x, train_y = sampled_x[:train_split], sampled_y[:train_split]

train_x, train_y = torch.from_numpy(train_x).float(), torch.from_numpy(train_y).float()
test_x, test_y = torch.from_numpy(test_x).float(), torch.from_numpy(test_y).float()

if normalize_features:
    mu_x, std_x = train_x.mean(dim=-2, keepdim=True), train_x.std(dim=-2, keepdim=True) + 1e-6
    train_x.sub_(mu_x).div_(std_x)
    test_x.sub_(mu_x).div_(std_x)
    
if normalize_labels:
    mu_y, std_y = train_y.mean(), train_y.std()
    train_y.sub_(mu_y).div_(std_y)
    test_y.sub_(mu_y).div_(std_y)

### Move Data to Device

In [5]:
train_x, train_y = train_x.contiguous(), train_y.contiguous()
test_x, test_y = test_x.contiguous(), test_y.contiguous()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

train_x, train_y = train_x.to(device), train_y.to(device)
test_x, test_y = test_x.to(device), test_y.to(device)

## Model

In [6]:
%%capture
likelihood = gpytorch.likelihoods.GaussianLikelihood(
    noise_constraint=gpytorch.constraints.GreaterThan(1e-8),
    noise_prior=None  # NormalPrior(torch.tensor([0.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

kernel = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=2.5))
# gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

model = VanillaGP(train_x, train_y, likelihood, kernel).to(device)

hypers = {
    'likelihood.noise_covar.noise': 1e-2,
    'covar_module.base_kernel.lengthscale': 1.0,
    'covar_module.outputscale': 1.0,
}
model.initialize(**hypers)

## Train

In [7]:
t0 = time()
model.vanilla_train(lr=1e-1, iter=100, verbose=True)
t1 = time()
print("Time: %.2g sec" % (t1 - t0))
torch.cuda.empty_cache()

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.86 GiB. GPU 0 has a total capacty of 23.64 GiB of which 2.61 GiB is free. Including non-PyTorch memory, this process has 20.08 GiB memory in use. Of the allocated memory 19.53 GiB is allocated by PyTorch, and 30.64 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Evaluation

In [ ]:
%%capture
likelihood.eval()
model.eval()

## Metrics

In [ ]:
with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cg_tolerance(10000):
    preds_test = likelihood(model(test_x))
        
    error = test_y - preds_test.mean
    covar = preds_test.lazy_covariance_matrix.evaluate_kernel()
    inv_quad, logdet = covar.inv_quad_logdet(inv_quad_rhs=error.unsqueeze(-1), logdet=True)
    
    rmse = (error.square().sum()/test_y.shape[0]).sqrt()
    nll = 0.5 * sum([inv_quad, logdet, error.size(-1)* np.log(2 * np.pi)])/test_y.shape[0]
    
print("RMSE: ", rmse)
print("NLL: ", nll)